## <b>4.3 PREPROCESSAMENT I ANÀLISI DE DADES</b>

### <b>4.3.2 Transformacions i enginyeria de variables bàsica</b>

#### <b>4.3.2.1 Preparació inicial i tractament de valors nuls</b>

In [4]:
# ============================================================
# Preparació inicial i tractament de valors nuls
# ============================================================
# Objectiu d'aquest script:
#   - Carregar el dataset net (clean_motor_insurance.csv) com a punt
#     de partida de la fase d’enginyeria bàsica.
#   - Verificar la integritat del conjunt de dades:
#        * dimensions (files, columnes)
#        * tipus de dades
#        * duplicats
#        * valors nuls
#   - Tractar els valors nuls segons la seva naturalesa:
#        * Nuls estructurals (Date_lapse):
#             - Crear Has_lapse (1 si hi ha baixa, 0 si no)
#             - Crear Is_active (1 si actiu, 0 si no)
#             - Derivar Policy_duration (només per pòlisses amb lapse)
#        * Nuls moderats (Length ~9.79%):
#             - Crear Length_missing_flag (indicador d’imputació)
#             - Imputar per mediana segmentada per Type_risk
#               (amb fallback a mediana global si un segment no té valors)
#        * Nuls baixos (Type_fuel ~1.67%):
#             - Imputar amb categoria 'Unknown' (no es fa one-hot aquí)
#
# Nota:
#   - Aquest subapartat NO crea encara les derivades temporals finals
#     (Driver_age, Vehicle_age, etc.) ni fa codificació avançada.
#   - El resultat és un dataframe `df_prep` lliure de nuls no estructurals,
#     amb variables bàsiques de control temporal i de qualitat.
# ============================================================

import pandas as pd
import numpy as np

# ------------------------------------------------------------
# 1) Càrrega del dataset net
# ------------------------------------------------------------
CLEAN_PATH = "clean_motor_insurance.csv"  # Ruta del fitxer net previ
# sep=None amb engine="python" deixa que pandas infereixi el separador
# (funciona bé si pot detectar si és ',' o ';', etc.)
df_prep = pd.read_csv(CLEAN_PATH, sep=None, engine="python")
print("clean_motor_insurance.csv carregat correctament")
print("Dimensions:", df_prep.shape)  # (n_files, n_columnes)

# ------------------------------------------------------------
# 2) Verificacions bàsiques d'integritat
# ------------------------------------------------------------
# 2.1 Duplicats exactes (files)
# .duplicated() marca True per files idèntiques a una anterior;
# .sum() compta quantes hi ha.
n_dup = df_prep.duplicated().sum()
print("\nDuplicats exactes:", n_dup)
# 2.2 Taula de nuls inicial
# isna().sum() → nº de nuls per columna
missing_count = df_prep.isna().sum()
# Calculem percentatge de nuls per columna
missing_pct = (missing_count / len(df_prep) * 100).round(2)
# Construïm taula amb recompte i percentatge de nuls
missing_table = (
    pd.DataFrame({"missing_count": missing_count, "missing_pct": missing_pct})
      .sort_values("missing_count", ascending=False)  # més nuls a dalt
)
print("\nTaula de nuls inicial (top 10):")
# display és útil en notebooks (Jupyter) per visualitzar bonic
display(missing_table.head(10))

# ------------------------------------------------------------
# 3) Tractament de nuls estructurals (Date_lapse)
# ------------------------------------------------------------
# Context: Date_lapse és NaN en pòlisses que continuen actives.
# Per tant, el NaN aquí no és "dada perduda", sinó informació:
# "no hi ha data de baixa" → pòlissa activa.
# 3.1 Has_lapse: 1 si hi ha data de baixa (no NaN), 0 si NaN
df_prep["Has_lapse"] = np.where(df_prep["Date_lapse"].isna(), 0, 1)
# 3.2 Is_active: complementari de Has_lapse
# Si Has_lapse = 1 (cancel·lada) → Is_active = 0
# Si Has_lapse = 0 (sense baixa) → Is_active = 1
df_prep["Is_active"] = 1 - df_prep["Has_lapse"]
# 3.3 Conversió robusta de Date_lapse i Date_start_contract a datetime
#     (si ja venien com datetime del fitxer net, no passa res)
df_prep["Date_lapse"] = pd.to_datetime(
    df_prep["Date_lapse"], format="%Y-%m-%d", errors="coerce"
)
df_prep["Date_start_contract"] = pd.to_datetime(
    df_prep["Date_start_contract"], format="%Y-%m-%d", errors="coerce"
)
# 3.4 Policy_duration: només per contractes finalitzats (Has_lapse=1)
# Calculem la durada (en anys) entre l’inici del contracte i la data de baixa.
# Si no hi ha lapse, deixem NaN (nuls estructurals).
df_prep["Policy_duration"] = np.where(
    df_prep["Has_lapse"] == 1,
    (df_prep["Date_lapse"] - df_prep["Date_start_contract"]).dt.days / 365.25,
    np.nan
).round(2)  # arrodonim a 2 decimals
# Validació ràpida del lapse
print("\nDistribució Has_lapse:")
display(df_prep["Has_lapse"].value_counts())

print(
    "Percentatge pòlisses cancel·lades:",
    round(df_prep["Has_lapse"].mean() * 100, 2), "%"
)

# ------------------------------------------------------------
# 4) Tractament de nuls baixos (Type_fuel)
# ------------------------------------------------------------
# Segons el disseny: imputar valors nuls amb categoria "Unknown".
# No es fa one-hot aquí; es deixa per fases posteriors de codificació.
if "Type_fuel" in df_prep.columns:
    # Comptem nuls abans d’imputar
    n_fuel_nulls_before = df_prep["Type_fuel"].isna().sum()
    # Substituïm NaN per la categoria "Unknown"
    df_prep["Type_fuel"] = df_prep["Type_fuel"].fillna("Unknown")
    # Verifiquem que ja no hi ha nuls
    n_fuel_nulls_after = df_prep["Type_fuel"].isna().sum()
    print("\nType_fuel nuls abans:", n_fuel_nulls_before)
    print("Type_fuel nuls després:", n_fuel_nulls_after)
    # Distribució en percentatges per veure pes d’"Unknown"
    print("Distribució Type_fuel:")
    display(df_prep["Type_fuel"].value_counts(normalize=True).rename("pct"))

# ------------------------------------------------------------
# 5) Tractament de nuls moderats (Length)
# ------------------------------------------------------------
# Estratègia:
#   - Crear flag Length_missing_flag (1 si s’ha imputat, 0 si original).
#   - Imputar nuls de Length segons mediana per Type_risk.
#   - Si un Type_risk no té valors no nuls, usar mediana global.
if "Length" in df_prep.columns:
    # 5.1 Flag de control: nuls actuals de Length
    df_prep["Length_missing_flag"] = np.where(df_prep["Length"].isna(), 1, 0)
    n_len_nulls_before = df_prep["Length"].isna().sum()
    print("\nLength nuls abans:", n_len_nulls_before)
    # 5.2 Mediana per segment Type_risk
    # groupby("Type_risk") calcula mediana de Length per cada grup de risc
    median_by_risk = df_prep.groupby("Type_risk")["Length"].median()
    # Mediana global per a fallback si algun Type_risk no té valors
    global_median = df_prep["Length"].median()
    # 5.3 Funció d’imputació segmentada fila a fila
    def impute_length(row):
        # Només imputem si el valor original és NaN
        if pd.isna(row["Length"]):
            # busquem la mediana del segment corresponent
            seg_med = median_by_risk.get(row["Type_risk"], np.nan)
            # si no hi ha mediana de segment, utilitzem la global
            return seg_med if not pd.isna(seg_med) else global_median
        # si no hi ha NaN, retornem el valor original
        return row["Length"]
    # Apliquem la funció a cada fila
    df_prep["Length"] = df_prep.apply(impute_length, axis=1)
    n_len_nulls_after = df_prep["Length"].isna().sum()
    print("Length nuls després:", n_len_nulls_after)
    # Control de descriptives després d’imputar
    print("\nDescriptives Length després de la imputació:")
    display(df_prep["Length"].describe())
    print(
        "Percentatge imputat (Length_missing_flag=1):",
        round(df_prep["Length_missing_flag"].mean() * 100, 2), "%"
    )

# ------------------------------------------------------------
# 6) Comprovació final de nuls no estructurals
# ------------------------------------------------------------
# Tornem a calcular nuls per verificar que només queden els esperats (estructurals)
missing_count_after = df_prep.isna().sum()
missing_pct_after = (missing_count_after / len(df_prep) * 100).round(2)
missing_table_after = (
    pd.DataFrame({
        "missing_count": missing_count_after,
        "missing_pct": missing_pct_after
    })
    .sort_values("missing_count", ascending=False)
)
print("\nTaula de nuls FINAL (top 10):")
display(missing_table_after.head(10))
# Variables on s’accepten nuls perquè són estructurals
structural_allowed = ["Date_lapse", "Policy_duration"]
# A partir de la taula de nuls final, eliminem les files que corresponen
# a variables estructurals per centrar-nos en la resta
non_struct_nulls = missing_table_after.drop(index=structural_allowed, errors="ignore")
# Validació final:
# No han de quedar nuls en cap variable que NO sigui estructural.
# Si en queden, llancem un error per detectar-ho aviat.
assert non_struct_nulls["missing_count"].sum() == 0, \
       "Hi ha nuls no estructurals pendents després del tractament!"
print("\nValidació correcta: només queden nuls estructurals esperats.")
print("Dimensions finals df_prep:", df_prep.shape)


clean_motor_insurance.csv carregat correctament
Dimensions: (105555, 34)

Duplicats exactes: 0

Taula de nuls inicial (top 10):


,missing_count,missing_pct
Date_lapse,70408,66.70
Length,10329,9.79
Type_fuel,1764,1.67
ID,0,0.00
Cylinder_capacity,0,0.00
Area,0,0.00
Second_driver,0,0.00
Year_matriculation,0,0.00
Power,0,0.00
N_doors,0,0.00



Distribució Has_lapse:


Has_lapse
0    70408
1    35147
Name: count, dtype: int64

Percentatge pòlisses cancel·lades: 33.3 %

Type_fuel nuls abans: 1764
Type_fuel nuls després: 0
Distribució Type_fuel:


...


Length nuls abans: 10329
Length nuls després: 0

Descriptives Length després de la imputació:


count    105555.000000
mean          4.185750
std           0.455467
min           1.978000
25%           3.941000
50%           4.202000
75%           4.433000
max           8.218000
Name: Length, dtype: float64

Percentatge imputat (Length_missing_flag=1): 9.79 %

Taula de nuls FINAL (top 10):


,missing_count,missing_pct
Policy_duration,70408,66.7
Date_lapse,70408,66.7
ID,0,0.0
Length,0,0.0
Year_matriculation,0,0.0
Power,0,0.0
Cylinder_capacity,0,0.0
Value_vehicle,0,0.0
N_doors,0,0.0
Type_fuel,0,0.0



Validació correcta: només queden nuls estructurals esperats.
Dimensions finals df_prep: (105555, 38)


#### <b>4.3.2.2 Derivació de variables temporals i coherència cronològica</b>

In [6]:
# ============================================================
# Derivació de variables temporals i coherència cronològica
# ============================================================
# Aquest script deriva i valida magnituds temporals clau a partir de
# pòlisses d’assegurança motor, ja pre-processades i carregades a df_prep.
# No aplica correccions, només genera flags, càlculs i un report.
# ============================================================

import pandas as pd  # Llibreria per manipulació de dades tabulars i tractament de dates
import numpy as np   # Llibreria numèrica usada aquí per a creació de columnes condicionals

# Copiem per no modificar df_prep original
df_time = df_prep.copy()

# ------------------------------------------------------------
# 0) FORÇAR CAST DE TOTES LES DATES A DATETIME
# ------------------------------------------------------------
# Garantim que totes les columnes de data siguin datetime64 encara que
# el fitxer anterior ja les hagués convertit.

date_cols = [
    "Date_start_contract",   # Inici del contracte
    "Date_last_renewal",     # Última renovació
    "Date_next_renewal",     # Renovació futura
    "Date_birth",            # Naixement conductor
    "Date_driving_licence",  # Obtenir carnet conduir
    "Date_lapse"             # Baixa/lapse (si n'hi ha)
]

for c in date_cols:
    # Com que les dates són ISO (YYYY-MM-DD), especifiquem dayfirst=False
    # per evitar warnings i garantir parsing coherent.
    df_time[c] = pd.to_datetime(
        df_time[c],
        format="%Y-%m-%d",   # especifiquem format ISO explícit
        errors="coerce"      # converteix valors no parsejables a NaT
    )

print("Tipus després de la conversió forçada:")
print(df_time[date_cols].dtypes)

# ------------------------------------------------------------
# 1) VARIABLES TEMPORALS DERIVADES
# ------------------------------------------------------------

# Edat del conductor (anys) a l'inici del contracte
df_time["Driver_age"] = (
    df_time["Date_start_contract"] - df_time["Date_birth"]
).dt.days / 365.25

# Antiguitat del carnet (anys) a la darrera renovació
df_time["Licence_age"] = (
    df_time["Date_last_renewal"] - df_time["Date_driving_licence"]
).dt.days / 365.25

# Antiguitat de la pòlissa (anys) fins la darrera renovació
df_time["Policy_age"] = (
    df_time["Date_last_renewal"] - df_time["Date_start_contract"]
).dt.days / 365.25

# Dies fins la pròxima renovació
df_time["Days_to_next"] = (
    df_time["Date_next_renewal"] - df_time["Date_last_renewal"]
).dt.days

# Antiguitat del vehicle en anys
df_time["Vehicle_age"] = (
    df_time["Date_start_contract"].dt.year - df_time["Year_matriculation"]
)

# ------------------------------------------------------------
# 2) DETECTAR ANOMALIES TEMPORALS
# ------------------------------------------------------------
# Validem inconsistències cronològiques i casos implausibles.

anomalies = {
    "birth_after_licence": (df_time["Date_birth"] > df_time["Date_driving_licence"]).sum(),
    # Naixement posterior a l'obtenció del carnet → impossible

    "licence_negative": (df_time["Licence_age"] < 0).sum(),
    # Antiguitat carnet negativa → renovació anterior a obtenció del carnet

    "policy_negative": (df_time["Policy_age"] < 0).sum(),
    # Renovació anterior a l'inici del contracte → inconsistència greu

    "next_before_last": (df_time["Days_to_next"] < 0).sum(),
    # Renovació futura anterior a la passada → error de cronologia

    "lapse_before_start": (
        df_time["Date_lapse"].notna() &
        (df_time["Date_lapse"] < df_time["Date_start_contract"])
    ).sum(),
    # Lapse abans de l'inici de contracte → inconsistent si és un sinistre/cancel·lació real

    "lapse_before_last": (
        df_time["Date_lapse"].notna() &
        (df_time["Date_lapse"] < df_time["Date_last_renewal"])
    ).sum()
    # Lapse abans de la darrera renovació → a revisar segons definició de negoci
}

print("\n--- ANOMALIES DETECTADES ---")
for k, v in anomalies.items():
    print(f"{k}: {v}")

# ------------------------------------------------------------
# 3) DESCRIPTIVES DE LES VARIABLES TEMPORALS
# ------------------------------------------------------------
# Revisem distribucions bàsiques per inspeccionar rangs i detectar valors extrems.

temp_vars = ["Driver_age", "Licence_age", "Policy_age", "Vehicle_age", "Days_to_next"]

print("\n--- DESCRIPTIVES VARIABLES TEMPORALS ---")
display(df_time[temp_vars].describe().T)

# ------------------------------------------------------------
# 4) PREVIEW FINAL
# ------------------------------------------------------------
# Inspecció visual de les primeres files amb totes les dates i derivades.

print("\nMostra de registres amb variables temporals derivades:")
display(df_time[[
    "Date_start_contract", "Date_last_renewal", "Date_next_renewal",
    "Date_birth", "Date_driving_licence", "Date_lapse",
    "Driver_age", "Licence_age", "Policy_age", "Vehicle_age", "Days_to_next"
]].head())

Tipus després de la conversió forçada:
Date_start_contract     datetime64[ns]
Date_last_renewal       datetime64[ns]
Date_next_renewal       datetime64[ns]
Date_birth              datetime64[ns]
Date_driving_licence    datetime64[ns]
Date_lapse              datetime64[ns]
dtype: object

--- ANOMALIES DETECTADES ---
birth_after_licence: 0
licence_negative: 31
policy_negative: 68
next_before_last: 0
lapse_before_start: 0
lapse_before_last: 404

--- DESCRIPTIVES VARIABLES TEMPORALS ---


,count,mean,std,min,25%,50%,75%,max
Driver_age,105555.0,44.344420,12.765212,18.015058,34.234086,43.972621,53.733060,91.529090
Licence_age,105555.0,24.765484,12.486053,-2.242300,14.410678,23.854894,34.171116,74.028747
Policy_age,105555.0,2.981346,3.923427,-0.911704,0.000000,1.002053,4.000000,37.998631
Vehicle_age,105555.0,9.244887,7.148871,0.000000,4.000000,9.000000,13.000000,68.000000
Days_to_next,105555.0,365.088020,0.283326,365.000000,365.000000,365.000000,365.000000,366.000000



Mostra de registres amb variables temporals derivades:


,Date_start_contract,Date_last_renewal,Date_next_renewal,Date_birth,Date_driving_licence,Date_lapse,Driver_age,Licence_age,Policy_age,Vehicle_age,Days_to_next
0,2015-11-05,2015-11-05,2016-11-05,1956-04-15,1976-03-20,NaT,59.556468,39.627652,0.000000,11,366
1,2015-11-05,2016-11-05,2017-11-05,1956-04-15,1976-03-20,NaT,59.556468,40.629706,1.002053,11,365
2,2015-11-05,2017-11-05,2018-11-05,1956-04-15,1976-03-20,NaT,59.556468,41.629021,2.001369,11,365
3,2015-11-05,2018-11-05,2019-11-05,1956-04-15,1976-03-20,NaT,59.556468,42.628337,3.000684,11,365
4,2017-09-26,2017-09-26,2018-09-26,1956-04-15,1976-03-20,NaT,61.448323,41.519507,0.000000,13,365


### <b>4.3.2.3 Tractament d'anomalies temporals i incoherències</b>

In [8]:
# ============================================================
# Tractament d'anomalies temporals i incoherències
# ============================================================
# Objectiu:
#   - Detectar i sanejar incoherències temporals residuals identificades
#     a la fase anterior (df_time).
#   - Crear flags de traçabilitat per a cada tipologia d’anomalia, per
#     poder traçar quins registres han estat corregits.
#   - Aplicar correccions conservadores (sense inventar informació nova),
#     només "retallant" les dates perquè no generin valors impossibles:
#        * Licence_age < 0:
#             -> ajustar Date_driving_licence = Date_last_renewal
#        * Policy_age  < 0:
#             -> ajustar Date_start_contract = Date_last_renewal
#        * Date_lapse < Date_last_renewal (si lapse):
#             -> ajustar Date_lapse = Date_last_renewal
#   - Recalcular les derivades temporals afectades després de canviar dates.
#   - Validar que no queden antiguitats negatives ni incoherències de lapse.
#
# Entrada:
#   df_time = dataset amb derivades temporals i anomalies detectades
#
# Sortida:
#   df_time_corr = dataset temporalment coherent + flags de control
# ============================================================

# Fem una còpia per no modificar df_time original
df_time_corr = df_time.copy()

# ------------------------------------------------------------
# 1) Flags de traçabilitat (abans de corregir)
# ------------------------------------------------------------
# Aquí només marquem quins registres són incoherents, sense tocar dades.
# Això permet:
#   - Analitzar quants i quins casos eren problemàtics.
#   - Mantenir rastre de què s’ha corregit després.
# 1.1 Llicència incoherent: Licence_age < 0
#     (equival a Date_driving_licence posterior a Date_last_renewal)
df_time_corr["Licence_incoherent_flag"] = np.where(
    df_time_corr["Licence_age"] < 0,  # condició d'incoherència
    1,                                 # valor si condició certa
    0                                  # valor si condició falsa
)
# 1.2 Pòlissa incoherent: Policy_age < 0
#     (equival a Date_start_contract posterior a Date_last_renewal)
df_time_corr["Policy_incoherent_flag"] = np.where(
    df_time_corr["Policy_age"] < 0,
    1,
    0
)
# 1.3 Lapse incoherent: Date_lapse abans de Date_last_renewal, però només
#     als registres on hi ha lapse (Date_lapse no és NaN).
df_time_corr["Lapse_incoherent_flag"] = np.where(
    (df_time_corr["Date_lapse"].notna()) &  # hi ha lapse
    (df_time_corr["Date_lapse"] < df_time_corr["Date_last_renewal"]),  # abans de last_renewal
    1,
    0
)
print("Flags d’incoherència (abans de corregir):")
# Sumem cada flag per veure quants casos hi ha de cada tipus
display(df_time_corr[
    ["Licence_incoherent_flag", "Policy_incoherent_flag", "Lapse_incoherent_flag"]
].sum())

# ------------------------------------------------------------
# 2) Correccions conservadores de dates
# ------------------------------------------------------------
# Ara que ja tenim les flags, apliquem correccions mínimes per evitar
# valors temporals impossibles, però sense inventar dades noves.
# Sempre "portem" les dates incoherents fins a Date_last_renewal.
# 2.1 Correcció Licence_age negativa:
#     Si Licence_incoherent_flag = 1, vol dir que Licence_age < 0.
#     Ajustem Date_driving_licence = Date_last_renewal
#     ⇒ després de la correcció, Licence_age = 0 anys.
mask_lic = df_time_corr["Licence_incoherent_flag"] == 1
df_time_corr.loc[mask_lic, "Date_driving_licence"] = df_time_corr.loc[mask_lic, "Date_last_renewal"]
# 2.2 Correcció Policy_age negativa:
#     Si Policy_incoherent_flag = 1, vol dir que Policy_age < 0.
#     Ajustem Date_start_contract = Date_last_renewal
#     ⇒ Policy_age passa a ser 0 anys.
mask_pol = df_time_corr["Policy_incoherent_flag"] == 1
df_time_corr.loc[mask_pol, "Date_start_contract"] = df_time_corr.loc[mask_pol, "Date_last_renewal"]
# 2.3 Correcció lapse incoherent:
#     Si Lapse_incoherent_flag = 1, vol dir que Date_lapse < Date_last_renewal.
#     Ajustem Date_lapse = Date_last_renewal per garantir que la data
#     de baixa no sigui anterior a l’última renovació.
mask_lap = df_time_corr["Lapse_incoherent_flag"] == 1
df_time_corr.loc[mask_lap, "Date_lapse"] = df_time_corr.loc[mask_lap, "Date_last_renewal"]

# ------------------------------------------------------------
# 3) Recalcular derivades temporals després de corregir
# ------------------------------------------------------------
# Després de modificar dates, cal recalcular totes les magnituds
# temporals derivades que depenen d’elles.

# Edat del conductor a l'inici del contracte
df_time_corr["Driver_age"] = (
    (df_time_corr["Date_start_contract"] - df_time_corr["Date_birth"]).dt.days / 365.25
)
# Antiguitat del carnet a la darrera renovació
df_time_corr["Licence_age"] = (
    (df_time_corr["Date_last_renewal"] - df_time_corr["Date_driving_licence"]).dt.days / 365.25
)
# Antiguitat de la pòlissa a la darrera renovació
df_time_corr["Policy_age"] = (
    (df_time_corr["Date_last_renewal"] - df_time_corr["Date_start_contract"]).dt.days / 365.25
)
# Dies fins la pròxima renovació
df_time_corr["Days_to_next"] = (
    (df_time_corr["Date_next_renewal"] - df_time_corr["Date_last_renewal"]).dt.days
)
# Antiguitat del vehicle a l'inici del contracte
df_time_corr["Vehicle_age"] = (
    df_time_corr["Date_start_contract"].dt.year - df_time_corr["Year_matriculation"]
)
# Si Policy_duration existeix i depèn de Date_start_contract i Date_lapse,
# també l’hem de recalcular amb les dates ja corregides:
df_time_corr["Policy_duration"] = np.where(
    df_time_corr["Has_lapse"] == 1,  # només té sentit per pòlisses cancel·lades
    (df_time_corr["Date_lapse"] - df_time_corr["Date_start_contract"]).dt.days / 365.25,
    np.nan  # per la resta, mantenim NaN com a nuls estructurals
).round(2)

# ------------------------------------------------------------
# 4) Validació post-correcció
# ------------------------------------------------------------
# Tornem a calcular alguns indicadors per verificar que les correccions
# han eliminat les incoherències temporals crítiques.
# Comptem quants registres continuen amb Licence_age < 0
lic_neg_after = (df_time_corr["Licence_age"] < 0).sum()
# Comptem quants registres continuen amb Policy_age < 0
pol_neg_after = (df_time_corr["Policy_age"] < 0).sum()
# Comptem quants lapses continuen sent incoherents (abans de Date_last_renewal)
lap_incoh_after = (
    (df_time_corr["Date_lapse"].notna()) &
    (df_time_corr["Date_lapse"] < df_time_corr["Date_last_renewal"])
).sum()
print("\n--- VALIDACIÓ POST-CORRECCIÓ ---")
print("Licence_age < 0 :", lic_neg_after)
print("Policy_age  < 0 :", pol_neg_after)
print("Lapse incoherent :", lap_incoh_after)

# ------------------------------------------------------------
# 5) Mostra de registres corregits (opcional)
# ------------------------------------------------------------
# Per revisar manualment què s’ha tocat, mostrem una mostra de files
# que tenien alguna incoherència (mask_lic, mask_pol o mask_lap).
print("\nMostra de registres corregits (Licence/Policy/Lapse):")
display(df_time_corr.loc[
    mask_lic | mask_pol | mask_lap,
    [
        "ID",                     # identificador de la pòlissa/registre
        "Date_start_contract",    # data d'inici (pot haver estat corregida)
        "Date_last_renewal",      # darrera renovació (referència de correcció)
        "Date_lapse",             # data de baixa (pot haver estat corregida)
        "Date_driving_licence",   # data del carnet (pot haver estat corregida)
        "Driver_age",             # derivades post-correcció
        "Licence_age",
        "Policy_age",
        "Licence_incoherent_flag",  # flags per saber què estava malament
        "Policy_incoherent_flag",
        "Lapse_incoherent_flag"
    ]
].head())

# df_time_corr queda llest per continuar amb:
#   - codificació categòrica bàsica
#   - variables de negoci derivades
#   - exportació final (ex: transformed_motor_insurance.csv)


Flags d’incoherència (abans de corregir):


Licence_incoherent_flag     31
Policy_incoherent_flag      68
Lapse_incoherent_flag      404
dtype: int64


--- VALIDACIÓ POST-CORRECCIÓ ---
Licence_age < 0 : 0
Policy_age  < 0 : 0
Lapse incoherent : 0

Mostra de registres corregits (Licence/Policy/Lapse):


,ID,Date_start_contract,Date_last_renewal,Date_lapse,Date_driving_licence,Driver_age,Licence_age,Policy_age,Licence_incoherent_flag,Policy_incoherent_flag,Lapse_incoherent_flag
268,136,2016-04-13,2017-04-13,2017-04-13,1990-04-26,45.147159,26.965092,0.999316,0,0,1
350,180,2015-09-15,2016-09-15,2016-09-15,1986-05-30,48.227242,30.297057,1.002053,0,0,1
371,192,2011-06-17,2016-06-17,2016-06-17,1994-12-01,53.127995,21.544148,5.002053,0,0,1
1454,726,2007-06-30,2017-06-30,2017-06-30,1990-01-01,53.801506,27.493498,10.001369,0,0,1
1729,861,2018-01-15,2018-01-15,NaT,2007-07-11,47.047228,10.516085,0.000000,0,1,0


### <b>4.3.2.4 Variables de negoci derivades</b>

In [10]:
# ============================================================
# Variables de negoci derivades
# ============================================================
# Objectiu:
#   - Crear atributs orientats a la perspectiva actuarial i econòmica.
#   - Afegir indicadors binaris de sinistralitat anual i històrica.
#   - Calcular la ràtio cost/prima: mesura clau de rendibilitat tècnica.
#   - Validar distribucions i coherència bàsica de les noves variables.
#
# Input esperat:
#   df_time_corr → dataset després de corregir incoherències temporals.
#
# Output:
#   df_business → dataset ampliat amb variables de negoci derivades.
# ============================================================

# Fem còpia del dataset corregit per no modificar df_time_corr directament
df_business = df_time_corr.copy()

# ------------------------------------------------------------
# 1) Indicador de sinistre anual
# ------------------------------------------------------------
# A partir del nombre de sinistres de l'any (`N_claims_year`),
# creem una variable binària:
#   - 1 si hi ha almenys un sinistre (N_claims_year > 0)
#   - 0 si no hi ha sinistres (N_claims_year = 0 o NaN tractat com 0 si es vol)
df_business["Has_claims_year"] = (df_business["N_claims_year"] > 0).astype(int)

# ------------------------------------------------------------
# 2) Indicador d’històric de sinistres
# ------------------------------------------------------------
# De manera anàloga, creem un indicador històric basat en `N_claims_history`:
#   - 1 si al llarg de l'històric hi ha hagut algun sinistre
#   - 0 si no hi ha sinistres registrats en la història
df_business["Has_claims_history"] = (df_business["N_claims_history"] > 0).astype(int)

# ------------------------------------------------------------
# 3) Ràtio econòmica cost/prima
# ------------------------------------------------------------
# Definim una ràtio econòmica clau: cost de sinistres anuals / prima anual.
# Aquesta ràtio:
#   - > 1 indica que el cost de sinistres supera la prima cobrada (pòlissa deficitària).
#   - < 1 indica marge tècnic positiu (prima cobreix sinistres).
#   - Si Premium = 0, la divisió dóna inf, que aquí convertim a NaN.
df_business["Claims_to_premium_ratio"] = (
    df_business["Cost_claims_year"] / df_business["Premium"]
).replace([np.inf, -np.inf], np.nan)  # evitem infinits si Premium=0

# ------------------------------------------------------------
# 4) Validació bàsica de les noves variables
# ------------------------------------------------------------

print("Distribució Has_claims_year:")
# value_counts() mostra quants 0 i 1 hi ha (sense normalitzar)
print(df_business["Has_claims_year"].value_counts())
print("\nDistribució Has_claims_history:")
print(df_business["Has_claims_history"].value_counts())
print("\nDescriptives Claims_to_premium_ratio:")
# .describe() dona info bàsica: count, mean, std, min, quartils i max
print(df_business["Claims_to_premium_ratio"].describe())

# ------------------------------------------------------------
# 5) Mostra de registres amb variables de negoci derivades
# ------------------------------------------------------------
# Mostrem una mostra de files amb les variables originals de sinistres,
# la prima i les noves derivades de negoci per fer una inspecció visual.
print("\nMostra de registres amb variables de negoci derivades:")
print(df_business[[
    "N_claims_year", "Has_claims_year",
    "N_claims_history", "Has_claims_history",
    "Premium", "Cost_claims_year", "Claims_to_premium_ratio"
]].head())


Distribució Has_claims_year:
Has_claims_year
0    85909
1    19646
Name: count, dtype: int64

Distribució Has_claims_history:
Has_claims_history
1    68959
0    36596
Name: count, dtype: int64

Descriptives Claims_to_premium_ratio:
count    105555.000000
mean          0.476172
std           4.720622
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max        1071.248039
Name: Claims_to_premium_ratio, dtype: float64

Mostra de registres amb variables de negoci derivades:
   N_claims_year  Has_claims_year  N_claims_history  Has_claims_history  \
0              0                0                 0                   0   
1              0                0                 0                   0   
2              0                0                 0                   0   
3              0                0                 0                   0   
4              0                0                 0                   0   

   Premium  Cost_claims_year  C

### <b>4.3.2.5 Validació final, exportació i mapa de features</b>

In [12]:
# ============================================================
# Validació final, exportació i mapa de features
# ============================================================
# Dataset d’entrada:
#   df_business → dataset final amb:
#       - nuls estructurals únicament a Date_lapse / Policy_duration
#       - variables temporals derivades
#       - incoherències corregides + flags de control
#       - variables de negoci derivades
#
# Sortides:
#   * transformed_motor_insurance.csv
#   * schema_after_4_3_2.csv
#   * Diccionari intern de mapa de features (feature_map)
# ============================================================

# Fem una còpia per no tocar df_business directament
df_final = df_business.copy()

# ------------------------------------------------------------
# 1) VALIDACIÓ FINAL DE NULS
# ------------------------------------------------------------
# Calculem quants nuls hi ha per variable al dataset final.
missing_table_final = df_final.isna().sum().to_frame("missing_count")
missing_table_final["missing_pct"] = (
    missing_table_final["missing_count"] / len(df_final) * 100
).round(2)
print("\n--- Nuls finals (top 10) ---")
# Mostrem les 10 variables amb més nuls (per inspecció visual)
display(missing_table_final.sort_values("missing_count", ascending=False).head(10))
# Nuls estructurals permesos (per definició de negoci):
#   - Date_lapse: només té valor quan hi ha cancel·lació (Has_lapse = 1).
#   - Policy_duration: només es pot calcular si Has_lapse = 1.
structural_allowed = ["Date_lapse", "Policy_duration"]
# Eliminem aquestes variables estructurals de la taula per analitzar només la resta
non_structural_nulls = missing_table_final.drop(index=structural_allowed, errors="ignore")
# Validació estricta:
# Comprovem que la suma de nuls en totes les altres variables sigui 0.
# Si no, llancem una excepció per aturar el pipeline.
assert non_structural_nulls["missing_count"].sum() == 0, \
       "Encara hi ha nuls no estructurals!"
print("\nValidació correcta: només queden nuls estructurals esperats.")

# ------------------------------------------------------------
# 2) ORDENACIÓ LÒGICA DE VARIABLES PER FAMÍLIES
# ------------------------------------------------------------
# Definim l’ordre de les columnes segons famílies conceptuals
# per facilitar lectura, debugging i modelització posterior.
# Identificador únic del registre/pòlissa
cols_id = ["ID"]
# Dates “crues” / originals del negoci
cols_dates = [
    "Date_start_contract", "Date_last_renewal", "Date_next_renewal",
    "Date_birth", "Date_driving_licence", "Date_lapse"
]
# Variables temporals derivades (antiguitats, durades, etc.)
cols_temporal_der = [
    "Driver_age", "Licence_age", "Vehicle_age", "Policy_age",
    "Days_to_next", "Policy_duration"
]
# Flags de qualitat i d’incoherències corregides
cols_flags = [
    "Has_lapse",
    "Licence_incoherent_flag", "Policy_incoherent_flag", "Lapse_incoherent_flag",
    "Length_missing_flag"
]
# Característiques físiques / tècniques del vehicle
cols_vehicle = [
    "Year_matriculation", "Power", "Cylinder_capacity",
    "Value_vehicle", "N_doors", "Length", "Weight"
]
# Variables de risc i perfil comercial
cols_risk = [
    "Distribution_channel", "Seniority", "Policies_in_force",
    "Max_policies", "Max_products", "Type_risk", "Area", "Second_driver"
]
# Variables de negoci i sinistralitat
cols_business = [
    "Premium", "N_claims_year", "Cost_claims_year",
    "N_claims_history", "R_Claims_history",
    "Has_claims_year", "Has_claims_history", "Claims_to_premium_ratio",
    "Payment", "Lapse", "Type_fuel"
]
# Juntem totes les columnes en l’ordre desitjat
all_cols = (
    cols_id
    + cols_dates
    + cols_temporal_der
    + cols_flags
    + cols_vehicle
    + cols_risk
    + cols_business
)
# Abans de reordenar, comprovem que totes aquestes columnes existeixen a df_final.
# Això ajuda a detectar errors de nom o passos previs que hagin fallat.
missing_in_final = [c for c in all_cols if c not in df_final.columns]
if missing_in_final:
    # Aturem amb un error explícit indicant quines columnes falten
    raise ValueError(f"Columnes esperades però absents a df_final: {missing_in_final}")
# Reordenem les columnes segons l’estructura definida
df_final = df_final[all_cols]
print("\nReordenació de variables completada.")
print("Dimensions df_final:", df_final.shape)

# ------------------------------------------------------------
# 3) GENERACIÓ DE L’ESQUEMA FINAL (schema_after_4_3_2.csv)
# ------------------------------------------------------------
# Es crea un esquema simple (diccionari de dades) amb:
#   - nom de cada variable
#   - tipus de dada (dtype) en pandas
schema = pd.DataFrame({
    "variable": df_final.columns,
    "dtype": df_final.dtypes.astype(str)
})
# Guardem aquest esquema a CSV per documentació i traçabilitat
schema.to_csv("schema_after_4_3_2.csv", index=False, encoding="utf-8")
print("\n📄 schema_after_4_3_2.csv generat correctament.")
display(schema.head(10))

# ------------------------------------------------------------
# 4) MAPA DE FEATURES PER FAMÍLIA DE MODELS
# ------------------------------------------------------------
# Definim de forma explícita quines variables utilitzarem
# en cada model (freqüència, severitat, etc.).
# Aquest mapa serveix com a diccionari intern o configuració.
feature_map = {
    # FREQÜÈNCIA (classificació Has_claims_year):
    # Variables que expliquen la probabilitat de tenir almenys un sinistre en l’any.
    "freq_model_features": [
        "Driver_age", "Licence_age", "Vehicle_age", "Has_lapse",
        "Policy_duration", "Second_driver", "Area", "Type_risk",
        "Type_fuel", "Has_claims_history", "Value_vehicle", "Power",
        "Premium", "Seniority", "Policies_in_force", "Distribution_channel"
    ],
    # SEVERITAT (regressió Cost_claims_year, condicionada a tenir sinistre):
    # Variables més orientades a quantificar el cost dels sinistres.
    "sev_model_features": [
        "Vehicle_age", "Value_vehicle", "Power", "Type_risk",
        "Area", "Policy_duration", "Claims_to_premium_ratio",
        "Weight", "Cylinder_capacity", "Length"
    ],
    # Flags de qualitat/traçabilitat (no entren necessàriament al model,
    # però serveixen per control i auditoria).
    "flags_features": cols_flags,
    # Variables objectiu (targets) per als models:
    #   - Freqüència: Has_claims_year (binària)
    #   - Severitat: Cost_claims_year (contínua)
    "targets": ["Has_claims_year", "Cost_claims_year"]
}
print("\n--- Mapa de features ---")
for k, v in feature_map.items():
    print(f"{k}: {v}")
# Convertim el mapa a un DataFrame per poder visualitzar-lo millor
mapa_features_df = pd.DataFrame(
    [(grp, var) for grp, vars_ in feature_map.items() for var in vars_],
    columns=["feature_group", "variable"]
)
display(mapa_features_df.head(20))

# ------------------------------------------------------------
# 5) EXPORTACIÓ DEL DATASET FINAL
# ------------------------------------------------------------
# Exportem el dataset final, llest per modelització o consum per altres eines.
df_final.to_csv("transformed_motor_insurance.csv", index=False, encoding="utf-8")
print("\n============================================================")
print("EXPORTACIÓ COMPLETADA")
print("Fitxers generats:")
print(" - transformed_motor_insurance.csv")
print(" - schema_after_4_3_2.csv")
print("============================================================")



--- Nuls finals (top 10) ---


,missing_count,missing_pct
Date_lapse,70408,66.7
Policy_duration,70408,66.7
ID,0,0.0
Value_vehicle,0,0.0
N_doors,0,0.0
Type_fuel,0,0.0
Length,0,0.0
Weight,0,0.0
Driver_age,0,0.0
Licence_age,0,0.0



Validació correcta: només queden nuls estructurals esperats.

Reordenació de variables completada.
Dimensions df_final: (105555, 44)

📄 schema_after_4_3_2.csv generat correctament.


,variable,dtype
ID,ID,int64
Date_start_contract,Date_start_contract,datetime64[ns]
Date_last_renewal,Date_last_renewal,datetime64[ns]
Date_next_renewal,Date_next_renewal,datetime64[ns]
Date_birth,Date_birth,datetime64[ns]
Date_driving_licence,Date_driving_licence,datetime64[ns]
Date_lapse,Date_lapse,datetime64[ns]
Driver_age,Driver_age,float64
Licence_age,Licence_age,float64
Vehicle_age,Vehicle_age,int64



--- Mapa de features ---
freq_model_features: ['Driver_age', 'Licence_age', 'Vehicle_age', 'Has_lapse', 'Policy_duration', 'Second_driver', 'Area', 'Type_risk', 'Type_fuel', 'Has_claims_history', 'Value_vehicle', 'Power', 'Premium', 'Seniority', 'Policies_in_force', 'Distribution_channel']
sev_model_features: ['Vehicle_age', 'Value_vehicle', 'Power', 'Type_risk', 'Area', 'Policy_duration', 'Claims_to_premium_ratio', 'Weight', 'Cylinder_capacity', 'Length']
flags_features: ['Has_lapse', 'Licence_incoherent_flag', 'Policy_incoherent_flag', 'Lapse_incoherent_flag', 'Length_missing_flag']
targets: ['Has_claims_year', 'Cost_claims_year']


,feature_group,variable
0,freq_model_features,Driver_age
1,freq_model_features,Licence_age
2,freq_model_features,Vehicle_age
3,freq_model_features,Has_lapse
4,freq_model_features,Policy_duration
5,freq_model_features,Second_driver
6,freq_model_features,Area
7,freq_model_features,Type_risk
8,freq_model_features,Type_fuel
9,freq_model_features,Has_claims_history



EXPORTACIÓ COMPLETADA
Fitxers generats:
 - transformed_motor_insurance.csv
 - schema_after_4_3_2.csv


#### <b>Rebaixar dataset a 4 decimals, decimals i milers</b>

In [14]:
# -------------------------------
# 1) Carregar dataset final
# -------------------------------
df = pd.read_csv("transformed_motor_insurance.csv")
# -------------------------------
# 2) Identificar columnes float
# -------------------------------
float_cols = df.select_dtypes(include="float").columns.tolist()
# -------------------------------
# 3) Arrodonir les float a 4 decimals
# -------------------------------
df[float_cols] = df[float_cols].round(4)
# -------------------------------
# 4) Convertir format numèric:
#    - sense separador de milers
#    - decimals amb coma ','
# -------------------------------
for col in float_cols:
    df[col] = (
        df[col]
        .astype(str)             # Convertir a text
        .str.replace('.', ',', regex=False)  # Substituir punt per coma
    )
# -------------------------------
# 5) Exportar CSV final net
# -------------------------------
output_path = "transformed_motor_insurance_rounded_european.csv"
df.to_csv(output_path, index=False, encoding="utf-8")
output_path


'transformed_motor_insurance_rounded_european.csv'